About knowledgebase;
e we create a simple JSON file that stores common issues and their solutions. It ensures that the knowledge base is structured in a way that allows easy retrieval of data. The code here shows a sample JSON which includes common issues such as slow internet or a device not charging. The knowledge base is a simple database that the agent will query when diagnosing user-reported problems. The more comprehensive the knowledge base, the more helpful the agent will be in troubleshooting. 

 Next, we'll implement the part of the agent that interacts with the user. Using natural language processing, or NLP, we can make the agent understand user queries and match them to problems in the knowledge base. 

1. Create a NEW environment

Install package

In [3]:
# Uninstall spacy and its dependency thinc
!pip uninstall -y spacy thinc

# Uninstall numpy to clear out the version causing the conflict
!pip uninstall -y numpy


Found existing installation: spacy 3.5.4
Uninstalling spacy-3.5.4:
  Successfully uninstalled spacy-3.5.4
Found existing installation: thinc 8.1.12
Uninstalling thinc-8.1.12:
  Successfully uninstalled thinc-8.1.12
Found existing installation: numpy 2.2.6
Uninstalling numpy-2.2.6:
  Successfully uninstalled numpy-2.2.6


In [3]:
import subprocess
import sys

# --- Recommended Fix for NumPy 2.x ABI Conflicts in Azure ML ---
# The environment's core libraries (TensorFlow, PyTorch, Scipy) are compiled 
# against NumPy 1.x. We must force-reinstall a compatible NumPy version 
# before importing the high-level NLP libraries.

print("Starting strict dependency rollback and reinstallation...")

# 1. Force downgrade/reinstall of core scientific libraries to 1.x compatible versions
#    We target known stable versions that predate the NumPy 2.0 shift.
core_libraries = [
    "numpy==1.23.5",  # Use a high-end 1.x version
    "scipy==1.10.1",
]

print(f"Installing core libraries: {', '.join(core_libraries)}")
try:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "--force-reinstall", *core_libraries])
    print("Core libraries successfully reinstalled.")
except subprocess.CalledProcessError as e:
    print(f"ERROR: Failed to install core libraries. {e}", file=sys.stderr)
    sys.exit(1)

# 2. Reinstall high-level NLP libraries and ensure spaCy model is downloaded
#    These will now link against the correct NumPy 1.x ABI.
nlp_libraries = [
    "nltk",
    "spacy",
    "transformers==4.40.2"
]

print(f"Installing NLP libraries: {', '.join(nlp_libraries)}")
try:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", *nlp_libraries])
    print("NLP libraries successfully reinstalled.")
except subprocess.CalledProcessError as e:
    print(f"ERROR: Failed to install NLP libraries. {e}", file=sys.stderr)
    sys.exit(1)


# 3. Download the required spaCy model (en_core_web_md)
print("Downloading required spaCy model (en_core_web_md)...")
try:
    subprocess.check_call([sys.executable, "-m", "spacy", "download", "en_core_web_md"])
    print("spaCy model download complete.")
except subprocess.CalledProcessError as e:
    print(f"ERROR: Failed to download spaCy model. {e}", file=sys.stderr)
    # Continue execution as the user can run the agent and see the error message if model is missing

print("\nDependency installation complete. Please restart your Python kernel now.")
print("After restarting, the 'troubleshooting_agent.py' script should run without the NumPy error.")

Starting strict dependency rollback and reinstallation...
Installing core libraries: numpy==1.23.5, scipy==1.10.1
  Using cached numpy-1.23.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Using cached scipy-1.10.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1
Core libraries successfully reinstalled.
Installing NLP libraries: nltk, spacy, transformers==4.40.2
NLP libraries successfully reinstalled.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 34.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
spaCy model download complete.

Dependency installation complete. P

/anaconda/envs/azureml_py38/lib/python3.10/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [1]:
import sys
import os

# --- CRITICAL FIX: Aggressively Block TensorFlow ---
# The previous fix was insufficient because transformers detects tensorflow on disk
# and tries to import it, causing a crash due to binary incompatibilities.
# We must trick Python's import system into thinking tensorflow is not installed.

# 1. Force 'tensorflow' to look like a missing module.
# This prevents any library from successfully importing it.
sys.modules['tensorflow'] = None

# 2. Clean up any existing transformers modules from memory.
# This ensures transformers re-initializes and sees our 'block' on tensorflow.
keys_to_remove = [k for k in sys.modules if k.startswith("transformers")]
for k in keys_to_remove:
    del sys.modules[k]

# 3. Set env vars as a secondary backup
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["TRANSFORMERS_NO_FLAX"] = "1"

import json
import spacy

# Import transformers *after* blocking tensorflow
from transformers import pipeline

# --- Knowledge Base (Integrated from JSON) ---
KNOWLEDGE_BASE = {
    "slow_internet": {
        "symptom": "My internet is slow.",
        "solution": "Restart your router and check your connection settings."
    },
    "software_crash": {
        "symptom": "The application keeps crashing.",
        "solution": "Update the application and restart your computer."
    },
    "no_sound": {
        "symptom": "No sound is coming from my speakers.",
        "solution": "Check if the speakers are connected and not muted."
    },
    "printer_not_working": {
        "symptom": "The printer is not working.",
        "solution": "Ensure the printer is connected and check for paper jams."
    },
    "device_not_charging": {
        "symptom": "My device is not charging.",
        "solution": "Check if the charger is properly connected and working."
    }
}

# --- Model Loading ---

# Load the spaCy medium model (requires word vectors for similarity)
try:
    # Use the medium model, which was downloaded in the setup step
    nlp = spacy.load("en_core_web_md")
except OSError:
    print("ERROR: 'en_core_web_md' model not found. Please ensure you ran the setup step to download it.", file=sys.stderr)
    sys.exit(1)

# Initialize a question-answering NLP pipeline
# This should now initialize successfully using the PyTorch backend.
try:
    qa_pipeline = pipeline("question-answering")
except Exception as e:
    print(f"ERROR: Could not initialize the Question Answering pipeline. Details: {e}", file=sys.stderr)
    # The agent can still function using the spaCy matching, so we continue.
    qa_pipeline = None


# --- Agent Functions ---

def find_best_match(user_input, knowledge_base):
    """
    Matches user input to the most relevant knowledge base entry using spaCy's similarity.
    """
    best_similarity = 0.0
    # Updated default response to guide the user on matching
    best_solution = "I couldn't find a direct solution in my knowledge base. Try being more descriptive."
    
    doc1 = nlp(user_input.lower()) # Process user input

    for issue, details in knowledge_base.items():
        symptom_text = details["symptom"]
        doc2 = nlp(symptom_text.lower()) # Process symptom text

        try:
            # Calculate similarity score (requires vectors in the spaCy model)
            similarity = doc1.similarity(doc2)
            
            # Update best match if current score is higher
            if similarity > best_similarity:
                best_similarity = similarity
                best_solution = details["solution"]
                
        except ValueError as e:
            # If similarity calculation fails (e.g., model vectors are missing), we skip the match
            print(f"Warning: Could not calculate similarity for '{issue}'. Model may lack word vectors.", file=sys.stderr)
            continue # Continue to the next item

    # Use a high threshold to determine a confident match
    if best_similarity > 0.8:
        return best_solution
    else:
        # Fallback if no strong match is found
        return best_solution # Return the best match found, even if it's not a strong fit

# Define a function to troubleshoot internet-related issues with follow-up questions
def diagnose_internet_issue():
    print("\n--- Diagnostic Assistant ---")
    print("The system detected a potential internet issue. Let's troubleshoot.")
    
    print("Have you tried restarting your router? This solves most connection issues.")
    response = input("Yes/No: ").strip().lower()
    
    if response == "no":
        print("ACTION: Please restart your router and check your connection stability.")
    else:
        print("The basic step failed. ACTION: Please try resetting your network settings manually or contacting your ISP.")

# --- Main Execution ---

print("Welcome to the Tech Troubleshooting Agent.")

# Gather user input from the console
user_input = input("Please describe your problem: ")

# 1. Find the best match in the knowledge base
solution = find_best_match(user_input, KNOWLEDGE_BASE)

# Output the solution
print(f"\nPotential Solution: {solution}")

# 2. Trigger diagnostic logic for internet issues (as requested in original code)
if "internet" in user_input.lower() or "slow connection" in user_input.lower():
    diagnose_internet_issue()

/anaconda/envs/azureml_py38/lib/python3.10/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/anaconda/envs/azureml_py38/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
/anaconda/envs/azureml_py38/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Welcome to the Tech Troubleshooting Agent.

Potential Solution: Restart your router and check your connection settings.

--- Diagnostic Assistant ---
The system detected a potential internet issue. Let's troubleshoot.
Have you tried restarting your router? This solves most connection issues.
The basic step failed. ACTION: Please try resetting your network settings manually or contacting your ISP.


Agen in automation fpr fix problem

In [2]:
import sys
import os
import subprocess # Import subprocess to allow running system commands for automation

# --- CRITICAL FIX: Aggressively Block TensorFlow ---
# The previous fix was insufficient because transformers detects tensorflow on disk
# and tries to import it, causing a crash due to binary incompatibilities.
# We must trick Python's import system into thinking tensorflow is not installed.

# 1. Force 'tensorflow' to look like a missing module.
# This prevents any library from successfully importing it.
sys.modules['tensorflow'] = None

# 2. Clean up any existing transformers modules from memory.
# This ensures transformers re-initializes and sees our 'block' on tensorflow.
keys_to_remove = [k for k in sys.modules if k.startswith("transformers")]
for k in keys_to_remove:
    del sys.modules[k]

# 3. Set env vars as a secondary backup
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["TRANSFORMERS_NO_FLAX"] = "1"

import json
import spacy

# Import transformers *after* blocking tensorflow
from transformers import pipeline

# --- Knowledge Base (Integrated from JSON) ---
KNOWLEDGE_BASE = {
    "slow_internet": {
        "symptom": "My internet is slow.",
        "solution": "Restart your router and check your connection settings."
    },
    "software_crash": {
        "symptom": "The application keeps crashing.",
        "solution": "Update the application and restart your computer."
    },
    "no_sound": {
        "symptom": "No sound is coming from my speakers.",
        "solution": "Check if the speakers are connected and not muted."
    },
    "printer_not_working": {
        "symptom": "The printer is not working.",
        "solution": "Ensure the printer is connected and check for paper jams."
    },
    "device_not_charging": {
        "symptom": "My device is not charging.",
        "solution": "Check if the charger is properly connected and working."
    }
}

# --- Model Loading ---

# Load the spaCy medium model (requires word vectors for similarity)
try:
    # Use the medium model, which was downloaded in the setup step
    nlp = spacy.load("en_core_web_md")
except OSError:
    print("ERROR: 'en_core_web_md' model not found. Please ensure you ran the setup step to download it.", file=sys.stderr)
    sys.exit(1)

# Initialize a question-answering NLP pipeline
# This should now initialize successfully using the PyTorch backend.
try:
    qa_pipeline = pipeline("question-answering")
except Exception as e:
    print(f"ERROR: Could not initialize the Question Answering pipeline. Details: {e}", file=sys.stderr)
    # The agent can still function using the spaCy matching, so we continue.
    qa_pipeline = None


# --- Agent Functions ---

def find_best_match(user_input, knowledge_base):
    """
    Matches user input to the most relevant knowledge base entry using spaCy's similarity.
    """
    best_similarity = 0.0
    # Updated default response to guide the user on matching
    best_solution = "I couldn't find a direct solution in my knowledge base. Try being more descriptive."
    
    doc1 = nlp(user_input.lower()) # Process user input

    for issue, details in knowledge_base.items():
        symptom_text = details["symptom"]
        doc2 = nlp(symptom_text.lower()) # Process symptom text

        try:
            # Calculate similarity score (requires vectors in the spaCy model)
            similarity = doc1.similarity(doc2)
            
            # Update best match if current score is higher
            if similarity > best_similarity:
                best_similarity = similarity
                best_solution = details["solution"]
                
        except ValueError as e:
            # If similarity calculation fails (e.g., model vectors are missing), we skip the match
            print(f"Warning: Could not calculate similarity for '{issue}'. Model may lack word vectors.", file=sys.stderr)
            continue # Continue to the next item

    # Use a high threshold to determine a confident match
    if best_similarity > 0.8:
        return best_solution
    else:
        # Fallback if no strong match is found
        return best_solution # Return the best match found, even if it's not a strong fit

def attempt_network_reset():
    """
    Attempts to reset the network manager service using system commands.
    Note: This requires sudo/root privileges and might fail in containerized environments.
    """
    print("\n[Auto-Fix] Attempting to reset network settings...")
    try:
        # We use subprocess to run the Linux command 'systemctl restart NetworkManager'
        # In a cloud environment like Azure ML, you might not have 'sudo' access,
        # so this is wrapped in a try/except block to handle failure gracefully.
        subprocess.run(["sudo", "systemctl", "restart", "NetworkManager"], check=True)
        print("[Auto-Fix] Network settings reset successfully.")
    except subprocess.CalledProcessError:
        print("[Auto-Fix] Error: Insufficient permissions to restart network service (Sudo required).")
        print("Manual Action: Please try restarting your router manually.")
    except FileNotFoundError:
        print("[Auto-Fix] Error: Network command not found. (Are you running on Linux?)")

# Define a function to troubleshoot internet-related issues with follow-up questions
def diagnose_internet_issue():
    print("\n--- Diagnostic Assistant ---")
    print("The system detected a potential internet issue. Let's troubleshoot.")
    
    print("Have you tried restarting your router? This solves most connection issues.")
    response = input("Yes/No: ").strip().lower()
    
    if response == "no":
        print("ACTION: Please restart your router and check your connection stability.")
        
        # New Feature: Offer to perform the fix automatically
        auto_fix = input("Would you like me to try resetting the network settings on this machine? (Yes/No): ").strip().lower()
        if auto_fix == "yes":
            attempt_network_reset()
            
    else:
        print("The basic step failed. ACTION: Please try resetting your network settings manually or contacting your ISP.")

# --- Main Execution ---

print("Welcome to the Tech Troubleshooting Agent.")

# Gather user input from the console
user_input = input("Please describe your problem: ")

# 1. Find the best match in the knowledge base
solution = find_best_match(user_input, KNOWLEDGE_BASE)

# Output the solution
print(f"\nPotential Solution: {solution}")

# 2. Trigger diagnostic logic for internet issues (as requested in original code)
if "internet" in user_input.lower() or "slow connection" in user_input.lower():
    diagnose_internet_issue()

Welcome to the Tech Troubleshooting Agent.

Potential Solution: Restart your router and check your connection settings.

--- Diagnostic Assistant ---
The system detected a potential internet issue. Let's troubleshoot.
Have you tried restarting your router? This solves most connection issues.
ACTION: Please restart your router and check your connection stability.


Failed to restart NetworkManager.service: Unit NetworkManager.service not found.
